In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-05 08:51:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-10-05 08:51:44 (200 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# First, let's see the distribution
user_ratings_count = df_ratings['user'].value_counts()
book_ratings_count = df_ratings['isbn'].value_counts()

print(f"Users with >= 200 ratings: {len(user_ratings_count[user_ratings_count >= 200])}")
print(f"Books with >= 100 ratings: {len(book_ratings_count[book_ratings_count >= 100])}")

# Filter users with less than 200 ratings
df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 200].index)]

# Filter books with less than 100 ratings
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_ratings_count[book_ratings_count >= 100].index)]

print("Filtered ratings shape:", df_ratings_filtered.shape)

# Merge books with ratings to include titles
df = pd.merge(df_ratings_filtered, df_books, on='isbn')

# Remove duplicates where the same user rated the same book multiple times
df = df.drop_duplicates(['user', 'title'])
print("After removing duplicates:", df.shape)

Users with >= 200 ratings: 905
Books with >= 100 ratings: 731
Filtered ratings shape: (49781, 3)
After removing duplicates: (49136, 5)


In [ ]:
# Create pivot table with books as rows and users as columns
book_ratings_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)
print("Pivot table shape:", book_ratings_pivot.shape)

# Convert to sparse matrix for efficient computation
book_ratings_matrix = csr_matrix(book_ratings_pivot.values)

Pivot table shape: (673, 888)


In [ ]:
# Build KNN model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
knn_model.fit(book_ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [ ]:
def get_recommends(book = ""):
    # Check if book exists in our dataset
    if book not in book_ratings_pivot.index:
        return [book, []]

    # Find the index of the book
    book_index = book_ratings_pivot.index.get_loc(book)

    # Find nearest neighbors
    distances, indices = knn_model.kneighbors(
        book_ratings_pivot.iloc[book_index, :].values.reshape(1, -1)
    )

    # Prepare the recommendations list
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        book_title = book_ratings_pivot.index[indices.flatten()[i]]
        distance = float(distances.flatten()[i])  # Convert to regular float
        recommended_books.append([book_title, distance])

    # Sort by distance in descending order (farthest first)
    recommended_books.sort(key=lambda x: x[1], reverse=True)

    return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
